# Glands Segmentator #


In this file, I use two models that I created to successfully perform gland segmentation. The process forms a pipeline where an input image is provided for segmentation, and the output is an image with the segmented glands.

# Procesing of big images #

Imports:

In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
from sklearn.cluster import AgglomerativeClustering
import tifffile

Loading models:

In [3]:
import unet_core 
print(unet_core.__file__)  # Pokazuje lokalizację faktycznego pliku
dir(unet_core)         



C:\Users\stszy\miniconda3\Lib\site-packages\unet_core\__init__.py


['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [4]:
import unet_core.unet_interface
print(dir(unet_core.unet_interface))


['Image', 'UNET', 'UNetArchitecture', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'cv2', 'get_binary_mask', 'get_contours_from_mask', 'np', 'prepare_image', 'torch', 'train_process']


C:\Users\stszy\AppData\Roaming\Python\Python312\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
import importlib.util
spec = importlib.util.find_spec("unet_core.unet_interface")
print(spec)


ModuleSpec(name='unet_core.unet_interface', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001699CD32150>, origin='C:\\Users\\stszy\\miniconda3\\Lib\\site-packages\\unet_core\\unet_interface.py')


In [6]:
detectionModel = YOLO("../model/saved_models/Glands_Finder_Augumented_Data_best.pt")
from unet_core.unet_interface import UNET
modelUNET=UNET(img_height=50, img_width=50)
modelUNET.set_model(in_channels=3, out_channels=1, name="C:/Users/stszy/miniconda3/Lib/site-packages/unet_core/GlandsFinder")


Chosen model successfully found
=> Loading checkpoint


 Helper function:

In [32]:
def tile_image(image, tile_size, overlap):
    tiles = []
    positions = []
    h, w = image.shape[:2]
    stride = tile_size - overlap

    for y in range(0, h, stride):
        for x in range(0, w, stride):
            x_end = min(x + tile_size, w)
            y_end = min(y + tile_size, h)
            tile = image[y:y_end, x:x_end]
            tiles.append(tile)
            positions.append((x, y))
    return tiles, positions


def detect_on_tiles(tiles, positions,model):
    all_detections = []
    for tile, (x_offset, y_offset) in zip(tiles, positions):
        results = model(tile, conf=0.6)[0]
        for box in results.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            x1 += x_offset
            x2 += x_offset
            y1 += y_offset
            y2 += y_offset
            conf = float(box.conf)
            cls = int(box.cls)
            all_detections.append((x1, y1, x2, y2, conf, cls))
    return all_detections


def draw_detections(image, detections, class_names):
    annotated = image.copy()
    for x1, y1, x2, y2, conf, cls in detections:
        cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        label = f"{class_names[cls]} {conf:.2f}"
        cv2.putText(annotated, label, (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return annotated

import math

def merge_overlapping_boxes(
    detections,
    merge_distance=80,
    size_similarity_thresh=0.3
):
    merged = []
    used = [False] * len(detections)

    for i, det in enumerate(detections):
        if used[i]:
            continue

        x1, y1, x2, y2, conf, cls = det
        w1 = x2 - x1
        h1 = y2 - y1
        cx1 = (x1 + x2) / 2
        cy1 = (y1 + y2) / 2
        group = [(x1, y1, x2, y2, conf)]
        used[i] = True

        for j in range(i + 1, len(detections)):
            if used[j]:
                continue

            x1_b, y1_b, x2_b, y2_b, conf_b, cls_b = detections[j]
            w2 = x2_b - x1_b
            h2 = y2_b - y1_b
            cx2 = (x1_b + x2_b) / 2
            cy2 = (y1_b + y2_b) / 2

            # 1. Środki blisko siebie
            dist = math.hypot(cx1 - cx2, cy1 - cy2)
            if dist > merge_distance:
                continue

            # 2. Jeden z wymiarów podobny
            width_similar = abs(w1 - w2) / max(w1, w2) <= size_similarity_thresh
            height_similar = abs(h1 - h2) / max(h1, h2) <= size_similarity_thresh
            if not (width_similar or height_similar):
                continue

            # Pasuje — dodaj do grupy
            group.append((x1_b, y1_b, x2_b, y2_b, conf_b))
            used[j] = True

        # Scal grupę
        xs1, ys1, xs2, ys2, confs = zip(*group)
        merged_box = (
            min(xs1),
            min(ys1),
            max(xs2),
            max(ys2),
            sum(confs) / len(confs),
            cls
        )
        merged.append(merged_box)

    return merged


import random

def draw_detections_varied_colors(image, detections, class_names, thickness=2, font_scale=0.5):
    annotated = image.copy()
    random.seed(42)

    for idx, det in enumerate(detections):
        x1, y1, x2, y2, conf, cls_id = det

        color = tuple(int(c) for c in np.random.randint(0, 256, size=3))

        label = f"{class_names[int(cls_id)]} {conf:.2f}"

        cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
        text_y = int(y1) - 5 if y1 > 10 else int(y1) + 15
        cv2.putText(
            annotated,
            label,
            (int(x1), text_y),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            color,
            thickness=1,
            lineType=cv2.LINE_AA
        )

    return annotated


In [33]:


image_path = "../preprocessedData/tissue_regions/1M01/tissue_region_0.tiff"
image = tifffile.imread(image_path)
if image.ndim == 3 and image.shape[2] > 3:
    image = image[:, :, :3]

if image.ndim == 2:
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

tile_size = 2048 
overlap = 1024
tiles, positions = tile_image(image, tile_size, overlap)

detections = detect_on_tiles(tiles, positions , detectionModel)

detections = merge_overlapping_boxes(detections, merge_distance=80)

class_names = detectionModel.names

annotated = draw_detections(image, detections, class_names)

cv2.imwrite("wynik_z_detections.png", annotated)

annotated =draw_detections_varied_colors(image, detections, class_names)
cv2.imwrite("wynik_z_detections_varied_colors.png", annotated)



0: 640x640 (no detections), 94.1ms
Speed: 3.6ms preprocess, 94.1ms inference, 65.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 90.6ms
Speed: 4.1ms preprocess, 90.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 109.9ms
Speed: 3.7ms preprocess, 109.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 93.4ms
Speed: 4.2ms preprocess, 93.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 82.1ms
Speed: 3.3ms preprocess, 82.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 75.4ms
Speed: 3.4ms preprocess, 75.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 78.9ms
Speed: 3.3ms preprocess, 78.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 gland, 81.6ms
Speed: 3.4ms preprocess, 81.6ms infere

True

True

In [39]:
import numpy as np

def tile_image(image, tile_size, overlap):
    tiles = []
    h, w = image.shape[:2]
    stride = tile_size - overlap

    ys = list(range(0, h, stride))
    xs = list(range(0, w, stride))

    n_tiles_y = len(ys)
    n_tiles_x = len(xs)

    positions_matrix = np.empty((n_tiles_y, n_tiles_x), dtype=object)

    for i, y in enumerate(ys):
        for j, x in enumerate(xs):
            x_end = min(x + tile_size, w)
            y_end = min(y + tile_size, h)
            tile = image[y:y_end, x:x_end]
            tiles.append(tile)
            positions_matrix[i, j] = (x, y)

    return tiles, positions_matrix


In [42]:
import numpy as np

def detect_on_tiles(tiles, positions_matrix, model):
    n_tiles_y, n_tiles_x = positions_matrix.shape
    detections_matrix = np.empty((n_tiles_y, n_tiles_x), dtype=object)

    tile_idx = 0  
    for i in range(n_tiles_y):
        for j in range(n_tiles_x):
            tile = tiles[tile_idx]
            x_offset, y_offset = positions_matrix[i, j]
            results = model(tile, conf=0.6)[0]

            detections = []
            for box in results.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                x1 += x_offset
                x2 += x_offset
                y1 += y_offset
                y2 += y_offset
                conf = float(box.conf)
                cls = int(box.cls)
                detections.append((x1, y1, x2, y2, conf, cls))

            detections_matrix[i, j] = detections
            tile_idx += 1

    return detections_matrix

def merge_boxes_from_matrix(detections_matrix, pixel_tol=10, rel_tol=0.02):
    """
    Scal detekcje z macierzy w jedną listę boxów.
    
    Args:
        detections_matrix: np.ndarray 2D z listami boxów [(x1,y1,x2,y2,conf,cls), ...]
        pixel_tol: tolerancja w pikselach do porównania krawędzi boxów
        rel_tol: tolerancja względna (np. 0.1 = 10%) do porównania wymiarów boxów
        
    Returns:
        merged_boxes: lista scalonych boxów (x1, y1, x2, y2, conf, cls)
    """

    # Spłaszczamy wszystkie detekcje do jednej listy
    all_boxes = []
    for row in detections_matrix:
        for boxes in row:
            all_boxes.extend(boxes)

    merged = []

    def boxes_should_merge(b1, b2):
        # b1 i b2 to boxy: (x1,y1,x2,y2,conf,cls)
        if b1[5] != b2[5]:
            return False  # różne klasy, nie łączymy

        # Sprawdzamy podobieństwo krawędzi:
        # porównujemy czy ich krawędzie są blisko siebie (pixel_tol)
        # lub różnice szerokości/wysokości są mniejsze niż rel_tol

        x1_1, y1_1, x2_1, y2_1 = b1[:4]
        x1_2, y1_2, x2_2, y2_2 = b2[:4]

        width_1 = x2_1 - x1_1
        width_2 = x2_2 - x1_2
        height_1 = y2_1 - y1_1
        height_2 = y2_2 - y1_2

        # Czy krawędzie X1 są blisko?
        x1_close = abs(x1_1 - x1_2) <= pixel_tol or abs(x1_1 - x1_2) <= rel_tol * max(width_1, width_2)
        # Czy krawędzie X2 są blisko?
        x2_close = abs(x2_1 - x2_2) <= pixel_tol or abs(x2_1 - x2_2) <= rel_tol * max(width_1, width_2)
        # Czy krawędzie Y1 są blisko?
        y1_close = abs(y1_1 - y1_2) <= pixel_tol or abs(y1_1 - y1_2) <= rel_tol * max(height_1, height_2)
        # Czy krawędzie Y2 są blisko?
        y2_close = abs(y2_1 - y2_2) <= pixel_tol or abs(y2_1 - y2_2) <= rel_tol * max(height_1, height_2)

        # Obiekty uznajemy za scalone jeśli:
        # - krawędzie w poziomie się pokrywają (x1 lub x2 są blisko)
        # - lub krawędzie w pionie się pokrywają (y1 lub y2 są blisko)

        horizontal_overlap = x1_close or x2_close
        vertical_overlap = y1_close or y2_close

        return horizontal_overlap or vertical_overlap

    # Scalanie boxów iteracyjne (dopóki można scalić coś nowego)
    changed = True
    while changed:
        changed = False
        new_merged = []
        used = [False] * len(all_boxes)

        for i in range(len(all_boxes)):
            if used[i]:
                continue
            box_a = all_boxes[i]
            merged_box = box_a
            for j in range(i + 1, len(all_boxes)):
                if used[j]:
                    continue
                box_b = all_boxes[j]
                if boxes_should_merge(merged_box, box_b):
                    # scal boxy
                    x1 = min(merged_box[0], box_b[0])
                    y1 = min(merged_box[1], box_b[1])
                    x2 = max(merged_box[2], box_b[2])
                    y2 = max(merged_box[3], box_b[3])
                    conf = max(merged_box[4], box_b[4])
                    cls = merged_box[5]  # obie klasy takie same

                    merged_box = (x1, y1, x2, y2, conf, cls)
                    used[j] = True
                    changed = True

            used[i] = True
            new_merged.append(merged_box)

        all_boxes = new_merged

    return all_boxes


In [45]:


image_path = "../preprocessedData/tissue_regions/1M01/tissue_region_0.tiff"
image = tifffile.imread(image_path)
if image.ndim == 3 and image.shape[2] > 3:
    image = image[:, :, :3]

if image.ndim == 2:
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

tile_size = 2048 
overlap = 1024

class_names = detectionModel.names
# 1. Podziel obraz na kafelki i pozycje macierzowe
tiles, positions_matrix = tile_image(image, tile_size, overlap)

# 2. Detekcja na kafelkach — wynik to macierz detekcji (lista list) o takim samym kształcie jak positions_matrix
detections_matrix = detect_on_tiles(tiles, positions_matrix, detectionModel)

# 3. Scal detekcje w jedną listę
merged_detections = merge_boxes_from_matrix(detections_matrix, pixel_tol=0, rel_tol=0.00)

# 4. Narysuj wykrycia na obrazie
all_boxes = draw_detections(image, merged_detections, class_names)
cv2.imwrite("wynik_z_detections.png", all_boxes)

annotated = draw_detections_varied_colors(image, merged_detections, class_names)
cv2.imwrite("wynik_z_detections_varied_colors.png", annotated)



0: 640x640 (no detections), 78.6ms
Speed: 3.5ms preprocess, 78.6ms inference, 31.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 80.7ms
Speed: 3.3ms preprocess, 80.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 82.1ms
Speed: 3.4ms preprocess, 82.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 82.8ms
Speed: 3.4ms preprocess, 82.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 84.6ms
Speed: 3.6ms preprocess, 84.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 74.2ms
Speed: 3.5ms preprocess, 74.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 82.0ms
Speed: 3.1ms preprocess, 82.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 gland, 82.9ms
Speed: 3.4ms preprocess, 82.9ms inferenc

True